In [2]:
import redis

# Create a connection pool
# Parameters are similar to Redis() constructor
pool = redis.ConnectionPool(
    host='srv01.dev.keycenter.ai',
    port=6379,
    # password='your_strong_password_here', # Change this if password is required
    db=0,
    decode_responses=True
)

try:
    # Get a connection from the pool
    r = redis.Redis(connection_pool=pool)

    r.ping()
    print("Connected to Redis via connection pool successfully!")

    # Perform operations
    r.set('pooled_key', 'Data from a pooled connection')
    print(f"Value of 'pooled_key': {r.get('pooled_key')}")

except redis.exceptions.ConnectionError as e:
    print(f"Could not connect to Redis: {e}")
except redis.exceptions.AuthenticationError as e:
    print(f"Authentication failed for Redis: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Connected to Redis via connection pool successfully!
Value of 'pooled_key': Data from a pooled connection


In [ ]:
import redis
import json
import os 
from dotenv import load_dotenv
# Load environment variables from .env file 

load_dotenv()

# --- Configuration for your Redis instance ---
REDIS_HOST = os.getenv('REDIS_HOST', 'localhost')
REDIS_PORT = int(os.getenv('REDIS_PORT', 6379))
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD', None)
REDIS_DB = 0
print(f"Connecting to Redis at {REDIS_HOST}:{REDIS_PORT}...")

try:
    r = redis.Redis(
        host=REDIS_HOST,
        port=REDIS_PORT,
        password=REDIS_PASSWORD,
        db=REDIS_DB,
        decode_responses=True # Decodes responses to strings automatically
    )
    r.ping()
    print(f"Connected to Redis at {REDIS_HOST}:{REDIS_PORT} successfully!")

except redis.exceptions.ConnectionError as e:
    print(f"Could not connect to Redis: {e}")
    exit() # Exit if no connection
except redis.exceptions.AuthenticationError as e:
    print(f"Authentication failed: {e}")
    exit()

Connecting to Redis at srv01.dev.keycenter.ai:6379...
Connected to Redis at srv01.dev.keycenter.ai:6379 successfully!


In [4]:
# --- Constants ---
BASE_KEY_PATH = "hospitality_chatbot_history"
MAX_HISTORY_LENGTH = 10 # Define how many messages to keep

# --- Functions to manage chat history ---

def get_chat_history_key(user_id: str, history_id: str) -> str:
    """Constructs the Redis key for a specific chat history."""
    return f"{BASE_KEY_PATH}/{user_id}/{history_id}"

def add_message_to_history(user_id: str, history_id: str, message: dict):
    """
    Adds a single message to a user's chat history list and trims it
    to keep only the last MAX_HISTORY_LENGTH messages.
    Messages are stored as JSON strings within the list.
    """
    key = get_chat_history_key(user_id, history_id)
    message_json = json.dumps(message)

    # RPUSH adds to the right (tail) of the list.
    # This means the newest messages are at the end.
    r.rpush(key, message_json)

    # LTRIM keeps elements from start_index to end_index (inclusive).
    # To keep the last N elements when new items are RPUSHed:
    # LTRIM key -N -1
    # -1 refers to the last element
    # -N refers to the Nth element from the end.
    r.ltrim(key, -MAX_HISTORY_LENGTH, -1)

    print(f"Added message to {key} and trimmed to {MAX_HISTORY_LENGTH} messages.")

def get_chat_history(user_id: str, history_id: str, count: int = -1) -> list:
    """
    Retrieves the chat history for a user.
    Returns messages in chronological order (oldest first).
    Since we're using RPUSH and LTRIM -N -1, the list is already in chronological order.
    """
    key = get_chat_history_key(user_id, history_id)
    # LRANGE 0 -1 gets all elements. If MAX_HISTORY_LENGTH is 10, it will return up to 10.
    # Use a specific count if you want fewer than the max history length.
    history_raw = r.lrange(key, 0, count - 1 if count > 0 else -1)

    # Convert JSON strings back to Python dictionaries/objects
    return [json.loads(msg) for msg in history_raw]

def delete_chat_history(user_id: str, history_id: str):
    """Deletes a specific chat history for a user."""
    key = get_chat_history_key(user_id, history_id)
    r.delete(key)
    print(f"Deleted chat history: {key}")

def get_all_user_chat_history_keys(user_id: str) -> list:
    """
    Finds all chat history keys for a given user.
    WARNING: KEYS command can be slow on large production databases.
    Consider using SCAN in production for better performance, or
    maintaining a separate SET of history_ids.
    """
    pattern = f"{BASE_KEY_PATH}/{user_id}/*"
    return r.keys(pattern)

# --- Example Usage ---

user_id_1 = "user123_last10"
chat_session_1 = "session_fixed_length"

print(f"\n--- Adding more than {MAX_HISTORY_LENGTH} messages ---")
for i in range(1, 15): # Add 14 messages
    message = {"role": "user" if i % 2 != 0 else "assistant", "content": f"Message {i}"}
    add_message_to_history(user_id_1, chat_session_1, message)

print(f"\n--- Retrieving History (should be max {MAX_HISTORY_LENGTH} messages) ---")
history = get_chat_history(user_id_1, chat_session_1)
print(f"\nHistory for {get_chat_history_key(user_id_1, chat_session_1)} (length: {len(history)}):")
for msg in history:
    print(f"  {msg['role']}: {msg['content']}")

# Add one more message
print("\n--- Adding one more message ---")
add_message_to_history(user_id_1, chat_session_1, {"role": "user", "content": "This is the very last message!"})

print(f"\n--- Retrieving History again (still max {MAX_HISTORY_LENGTH} messages) ---")
history_after_add = get_chat_history(user_id_1, chat_session_1)
print(f"\nHistory for {get_chat_history_key(user_id_1, chat_session_1)} (length: {len(history_after_add)}):")
for msg in history_after_add:
    print(f"  {msg['role']}: {msg['content']}")

# print("\n--- Deleting History ---")
# delete_chat_history(user_id_1, chat_session_1)


--- Adding more than 10 messages ---
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimmed to 10 messages.
Added message to hospitality_chatbot_history/user123_last10/session_fixed_length and trimm